In [1]:
import pandas as pd
import nltk
import sddk

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [2]:
# for exporting data to googlesheets/sciencedata
# (feel free to skip)

s = sddk.cloudSession("sciencedata.dk")
# establish connection with gogglesheets...
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict") # or load it from a local storage: json.load(open("../../ServiceAccountsKey.json", "r"))
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
paul_results = gc.open_by_url("https://docs.google.com/spreadsheets/d/1h4M-gK9TPIfeTV528tUuPBfZF1wtcNCA10yIlJYqGTE/edit?usp=sharing")

endpoint variable has been configured to: https://sciencedata.dk/files/


In [3]:
LAGTec = pd.read_json("../data/large_files/LAGTec.json")

In [4]:
print(LAGTec["author"].unique().tolist())

['Gospel of Matthew', 'Gospel of Mark', 'Luke-Acts', 'Johnannine literature (New Testament)', 'Paul of Tarsus', 'Pauline literature', 'Hebrews (New Testament)', 'James (New Testament)', '1 Peter (New Testament)', '2 Peter (New Testament)', 'Jude (New Testament)', 'Revelation (New Testament)', 'Acta Joannis (recensio)', 'Clement of Alexandria', 'Maximus of Tyre', 'Justin Martyr', 'Pseudo-Justin Martyr', 'Athenagoras', 'Barnabas', 'Clement of Rome', 'Didache', 'Dionysius of Corinth', 'The Shepherd of Hermas', 'Ignatius of Antioch', 'Irenaeus', 'Martyrdom of Polycarp', 'Polycarp', 'Seniores Apud Irenaeum', 'Theophilus of Antioch', 'Tatianus', 'The Passion of Saints Perpetua and Felicity', 'Eusebius', 'Epiphanius', 'Gregory of Nazianzus', 'Pappus Alexandrinus', 'Athanasius of Alexandria', 'Basil of Caesarea', 'Origen', 'Sozomen', 'Socrates of Constantinople', 'Philostorgius', 'Hippolytus of Rome', 'Mark the Deacon', 'Marcianus of Heraclea', 'Marinus', 'Colluthus', 'Zosimus', 'Theodoret', '

In [5]:
LAGTec["sentences_paul_wordlist"] = LAGTec["sentences_paul"].apply(lambda x: [w for s in x for w in s])

In [6]:
paul_wordlist = [w for wordlist in LAGTec["sentences_paul_wordlist"] for w in wordlist]

In [7]:
paul_wordlist_freqs = nltk.FreqDist(paul_wordlist).most_common()
paul_wordlist_freqs[:50]

[('Παῦλος', 1072),
 ('λέγω', 380),
 ('εἰμί', 215),
 ('θεός', 198),
 ('φημί', 167),
 ('οὗτος', 134),
 ('γίγνομαι', 130),
 ('Χριστός', 128),
 ('ἀπόστολος', 123),
 ('λόγος', 103),
 ('Ἰησοῦς', 103),
 ('πᾶς', 90),
 ('ἄνθρωπος', 79),
 ('ἅγιος', 74),
 ('πολύς', 68),
 ('ἐκκλησία', 64),
 ('κύριος', 63),
 ('ἔχω', 61),
 ('πνεῦμα', 56),
 ('αὐτός', 56),
 ('Ἰουδαῖος', 55),
 ('Πέτρος', 52),
 ('ἐπίσκοπος', 48),
 ('γράφω', 47),
 ('ποιέω', 44),
 ('ἄλλος', 43),
 ('ἀκούω', 39),
 ('μόνος', 39),
 ('υἱός', 38),
 ('τὶς', 38),
 ('ἐπιστολή', 38),
 ('σοφία', 38),
 ('πατήρ', 37),
 ('διδάσκω', 35),
 ('πίστις', 34),
 ('ὁράω', 32),
 ('ἀδελφός', 32),
 ('νόμος', 32),
 ('ἀνήρ', 31),
 ('οἶδα', 31),
 ('εὐαγγέλιον', 31),
 ('σῶμα', 31),
 ('ἔθνος', 30),
 ('Κορίνθιος', 30),
 ('πρῶτος', 29),
 ('πόλις', 29),
 ('ἡμέρα', 29),
 ('ὄνομα', 28),
 ('μέλλω', 28),
 ('δύναμις', 28)]

In [8]:
paul_wordlist_freqs_df = pd.DataFrame(paul_wordlist_freqs, columns=["word", "count"])
paul_wordlist_freqs_df[:100]

,word,count
0,Παῦλος,1072
1,λέγω,380
2,εἰμί,215
3,θεός,198
4,φημί,167
...,...,...
95,παρίστημι,16
96,λοιπός,16
97,νοέω,16
98,Ἰωάν(ν)ης,16


In [9]:
len(paul_wordlist_freqs_df[paul_wordlist_freqs_df["count"]>=5])

396

In [10]:
set_with_dataframe(paul_results.worksheet("paul_sentences_wordfreqs"), paul_wordlist_freqs_df[paul_wordlist_freqs_df["count"]>=5])